<a href="https://colab.research.google.com/github/tpike3/mesa-geo/blob/newsphinx/docs/source/tutorials/intro_tutorial_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introductory Tutorial

This tutorial introduces Mesa-Geo, a GIS integrated Agent Based model platform that is part of the [Mesa ABM Ecosystem](https://mesa.readthedocs.io/en/stable/)

In this tutorial we will build a Geo-Schelling model, which takes the seminal [Schelling Segregation model](https://en.wikipedia.org/wiki/Schelling%27s_model_of_segregation) and replicates with the countries of Europe, were the countries "move" their preference similar to the Schelling Model.


In [ ]:
# Step 1: Install the package from the specified branch
!pip install git+https://github.com/tpike3/mesa-geo.git@newsphinx --quiet

# Step 2: Download the data file from the GitHub repository
!wget https://raw.githubusercontent.com/tpike3/mesa-geo/newsphinx/docs/source/tutorials/data/nuts_rg_60M_2013_lvl_2.geojson

In [ ]:
import random

import mesa
import mesa_geo as mg
import mesa_geo.visualization as mgv

## Create the Agents

The next cell creates the GeoAgents, in which European country is randomly assigned a preference of minority or majority and then if the agent is unhappy moves.  

In [ ]:
class SchellingAgent(mg.GeoAgent):
    """Schelling segregation agent."""

    def __init__(self, unique_id, model, geometry, crs, agent_type=None):
        """Create a new Schelling agent.

        Args:
            unique_id: Unique identifier for the agent.
            agent_type: Indicator for the agent's type (minority=1, majority=0)
        """
        super().__init__(unique_id, model, geometry, crs)
        self.atype = agent_type

    def step(self):
        """Advance agent one step."""
        similar = 0
        different = 0
        neighbors = self.model.space.get_neighbors(self)
        if neighbors:
            for neighbor in neighbors:
                if neighbor.atype is None:
                    continue
                elif neighbor.atype == self.atype:
                    similar += 1
                else:
                    different += 1

        # If unhappy, move:
        if similar < different:
            # Select an empty region
            empties = [a for a in self.model.space.agents if a.atype is None]
            # Switch atypes and add/remove from scheduler
            new_region = random.choice(empties)
            new_region.atype = self.atype
            self.model.schedule.add(new_region)
            self.atype = None
            self.model.schedule.remove(self)
        else:
            self.model.happy += 1

    def __repr__(self):
        return "Agent " + str(self.unique_id)

## Model Class

This class initiates the model class, which acts as a "manager" for the simulation, it holds the geo-spatial space. It holds the schedule of agents and their order, and the data collector to collect information from the model.

In [ ]:
class GeoSchelling(mesa.Model):
    """Model class for the Schelling segregation model."""

    def __init__(self, density=0.6, minority_pc=0.2, export_data=False):
        super().__init__()
        self.density = density
        self.minority_pc = minority_pc
        self.export_data = export_data

        self.schedule = mesa.time.RandomActivation(self)
        self.space = mg.GeoSpace(crs="EPSG:4326", warn_crs_conversion=True)

        self.happy = 0
        self.datacollector = mesa.DataCollector({"happy": "happy"})

        self.running = True

        # Set up the grid with patches for every NUTS region
        ac = mg.AgentCreator(SchellingAgent, model=self)
        agents = ac.from_file("nuts_rg_60M_2013_lvl_2.geojson")
        self.space.add_agents(agents)

        # Set up agents
        for agent in agents:
            if random.random() < self.density:
                if random.random() < self.minority_pc:
                    agent.atype = 1
                else:
                    agent.atype = 0
                self.schedule.add(agent)

    def export_agents_to_file(self) -> None:
        self.space.get_agents_as_GeoDataFrame(agent_cls=SchellingAgent).to_crs(
            "epsg:4326"
        ).to_file("schelling_agents.geojson", driver="GeoJSON")

    def step(self):
        """Run one step of the model.

        If All agents are happy, halt the model.
        """

        self.happy = 0  # Reset counter of happy agents
        self.schedule.step()
        self.datacollector.collect(self)

        if self.happy == self.schedule.get_agent_count():
            self.running = False

        if not self.running and self.export_data:
            self.export_agents_to_file()

## Build the Visual

This section of code set ups the conditions for drawing the agents and defining the model parameters. The next cell passes the `schelling_draw` function and model parameters into Mesa Geo's `GeoJupyterViz` so we can see a visual for our simulation.  

In [ ]:
def schelling_draw(agent):
    """
    Portrayal Method for canvas
    """
    portrayal = {}
    if agent.atype is None:
        portrayal["color"] = "Grey"
    elif agent.atype == 0:
        portrayal["color"] = "Orange"
    else:
        portrayal["color"] = "Blue"
    return portrayal


model_params = {
    "density": {
        "type": "SliderFloat",
        "value": 0.6,
        "label": "Population Density",
        "min": 0.0,
        "max": 0.9, #there must be an empty space for the agent to move
        "step": 0.1,
    },
    "minority_pc": {
        "type": "SliderFloat",
        "value": 0.2,
        "label": "Fraction Minority",
        "min": 0.0,
        "max": 1.0,
        "step": 0.05,
    },
    "export_data": {
        "type": "Checkbox",
        "value": False,
        "description": "Export Data",
        "disabled": False,
    },
}

## Run the Model

In [ ]:
page = mgv.GeoJupyterViz(
    GeoSchelling,
    model_params,
    measures=["happy"],
    name="Geo-Schelling Model",
    agent_portrayal=schelling_draw,
    zoom=3,
    center_point=[52, 12],
)
# This is required to render the visualization in the Jupyter notebook
page